<div style="text-align:center">
    <h1 style="font-size: 40px; font-weight: bold; color: orange;">WEB SCRAPING</h1>
</div>


<div>
    <h1 style="font-size: 14px; font-weight: bold; color: white;">BY:
    <br> TARUN RAVEESH - AM.EN.U4CSE20016
    <br> SHOAIB AKHTAR - AM.EN.U4CSE20163
    <br> CHETAN ANAND - AM.EN.U4CSE20219 </h1>
</div>


In [1]:
import pandas as pd
from random import randint
from selenium import webdriver
from time import sleep, strftime
from IPython.display import display
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
chromedriver_path = 'C:\\Users\\Tarun\\OneDrive\\Desktop\\Web Mining\\Chrome Driver\\chromedriver.exe'

In [ ]:
service = ChromeService(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service)

In [ ]:
URL = 'https://www.kayak.co.in/flights/COK-BOM/2023-10-20?sort=bestflight_a'
driver.get(URL)
sleep(3)

In [ ]:
# Explicitly wait for the pop-up close button to be clickable
try:
    close_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'bBPb-close'))
    )
    close_button.click()
except:
    print("Pop-up close button not found or couldn't be clicked")

In [ ]:
def more_flights(driver):
    try:
        show_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@class="ULvh-button show-more-button"]'))
        )
        show_more_button.click()
        print("More flights loaded successfully.")
    except:
        print("Show more results button not found or couldn't be clicked.")
more_flights(driver)

In [ ]:
def flight_data():
    # Define XPaths for the required fields
    xpath_airlines = "//div[@class='J0g6-operator-text']"
    xpath_departure_time = "//div[@class='vmXl vmXl-mod-variant-large']//span[1]"
    xpath_arrival_time = "//div[@class='vmXl vmXl-mod-variant-large']//span[3]"
    class_layovers = "JWEO-stops-text"
    class_duration = "xdW8-mod-full-airport"
    xpath_ticket_type = "//div[@class='aC3z-name']"
    xpath_price = "//div[@class='f8F1-price-text-container']/div[@class='f8F1-price-text']"
    
    # Use XPaths to extract data
    airlines_elements = driver.find_elements(By.XPATH, xpath_airlines)
    departure_times = driver.find_elements(By.XPATH, xpath_departure_time)
    arrival_times = driver.find_elements(By.XPATH, xpath_arrival_time)
    layovers = driver.find_elements(By.CLASS_NAME, class_layovers)
    durations = driver.find_elements(By.CLASS_NAME, class_duration)
    ticket_types = driver.find_elements(By.XPATH, xpath_ticket_type)
    prices = driver.find_elements(By.XPATH, xpath_price)

    # Initialize lists to store data
    airlines_list = []
    departure_times_list = [departure.text if departure.text else "N/A" for departure in departure_times]
    arrival_times_list = [arrival.text if arrival.text else "N/A" for arrival in arrival_times]
    layovers_list = []
    durations_list = []
    ticket_types_list = [ticket_type.text if ticket_type.text else "N/A" for ticket_type in ticket_types]
    prices_list = [price.text if price.text else "N/A" for price in prices]

    # Extract and format airline information
    for element in airlines_elements:
        airlines_text = element.text
        if airlines_text:
            airlines_list.append(airlines_text)

    # Process layovers and durations
    for layover in layovers:
        layover_text = layover.text.strip()
        if layover_text == "direct":
            layovers_list.append("Direct")
        else:
            layovers_list.append("Layover")

    for duration in durations:
        duration_text = duration.text.strip()
        durations_list.append(duration_text if duration_text else "N/A")

    # Create a DataFrame with the extracted data
    data = {
        'Airlines': airlines_list,
        'Departure Time': departure_times_list,
        'Arrival Time': arrival_times_list,
        'Layover': layovers_list,
        'Duration': durations_list,
        'Ticket Type': ticket_types_list,
        'Price': prices_list
    }

    df = pd.DataFrame(data)
    return df

In [ ]:
def scrape_flights(city_from, city_to, travel_date):
    """City codes - it's the IATA codes!
    Date format -  YYYY-MM-DD"""
    URL = ('https://www.kayak.co.in/flights/' + city_from + '-' + city_to +
             '/' + travel_date + '?sort=bestflight_a')
    driver.get(URL)
    sleep(randint(8, 10))
    
    # Sometimes a popup shows up, so we can use a try statement to check for it and close
    try:
        close_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'bBPb-close'))
        )
        close_button.click()
    except:
        pass
        
    sleep(randint(20,25))
    
    # Best Flights
    print('Fetching Best Flight Results.....')
    print('Starting First Scrape.....')
    df_flights_best = flight_data()
    df_flights_best['Sort'] = 'Best Flights'
    display(df_flights_best.head(3))
    sleep(randint(20,25))
    
    # Quickest Flights
    print('Switching to Quickest Flight Results.....')
    quick_results = driver.find_element(By.XPATH, '//div[@aria-label="Quickest"]')
    quick_results.click()
    sleep(randint(20,25))
    
    print('Starting Second Scrape.....')
    df_flights_fast = flight_data()
    df_flights_fast['Sort'] = 'Fast Flights'
    display(df_flights_fast.head(3))
    sleep(randint(20,25))
    
    # Cheapest Flights
    print('Switching to Cheapest Flight Results.....')
    cheap_results = driver.find_element(By.XPATH, '//div[@aria-label="Cheapest"]')
    cheap_results.click()
    sleep(randint(20,25))
    
    print('Starting Third Scrape.....')
    df_flights_cheap = flight_data()
    df_flights_cheap['Sort'] = 'Cheap Flights'
    display(df_flights_cheap.head(3))
    sleep(randint(20,25))
    
    
    # Saving a new dataframe as an excel file. the name is custom-made for your cities and dates
    final_df = pd.concat([df_flights_best, df_flights_cheap, df_flights_fast])
    final_df.to_excel('Data/{}_flights_{}-{}_for_{}.xlsx'.format(strftime("%Y%m%d-%H%M"), city_from,
                                                                    city_to, travel_date), index=False)
    print('Excel Generated')

In [ ]:
city_from = input('From which city? ')
city_to = input('Where to? ')
travel_date = input('Search around which departure date? Please use YYYY-MM-DD format only ')

scrape_flights(city_from, city_to, travel_date)
print('Scraping completed at {}'.format(strftime("%Y-%m-%d - %H:%M")))

In [ ]:
# Save a screenshot!
driver.save_screenshot('pythonscraping.png')

In [ ]:
driver.quit()